In [1]:
from llm import chat_openai
import numpy as np 
import json
import logging 
import matplotlib.pyplot as plt
import os
import openai
import re
import subprocess
from pathlib import Path
import shutil
import time 
import types
import copy

In [2]:
goal = '''
Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

Goal
It is your job to predict the sales price for each house. For each Id in the test set, you must predict the value of the SalePrice variable. 

Metric
Submissions are evaluated on Root-Mean-Squared-Error (RMSE) between the logarithm of the predicted value and the logarithm of the observed sales price. (Taking logs means that errors in predicting expensive houses and cheap houses will affect the result equally.)

Submission File Format
The file should contain a header and have the following format:

Id,SalePrice
1461,169000.1
1462,187724.1233
1463,175221
etc.
'''

In [3]:
root_file_directory = "agent_files"

In [4]:
exploration_progress = "None"

In [5]:
class CurriculumAgent():
    def __init__(self, goal="", memory=[], completed_tasks=[], failed_tasks=[], files="", saved_notes=""):
        self.goal = goal
        self.memory = memory
        self.completed_tasks = completed_tasks
        self.failed_tasks = failed_tasks
        self.files = files
        self.saved_notes = saved_notes

        # The crux is a Q&A process
        # Problem with this approach is you still have to deal with searching multiple times, and continuing to search or not. Approach: Or maybe if you search and you don't have the answer, that's a bad thing to search and you need to go more specific / ask a different question!
        self.system_prompt_automatic_curriculum = f'''You are a helpful assistant that asks questions to help me decide the next immediate question to answer on the computer. My ultimate goal is to discover as many useful pieces of information as possible, answer as many questions as possible, and become the best researcher in the world.

        Goal: {self.goal}

        I will give you the following information:
        Memory (oldest to newest): ...
        Recent links: ...
        Root folder inventory: ...
        Saved notes: ...
        Completed tasks so far: ...
        Failed tasks that are too hard: ...

        1) You should act as a mentor and guide me to the next question based on my current learning progress.
        2) Please be very specific about what question I need to answer.
        3) The next question should follow a clear format, such as "What do other papers say about [topic]", "What are the problems with [approach]", "How can I solve this [problem]", "Can results from [topic 1] be applied to [topic 2]?", "What are the similarities between [topic 1] for success and [topic 2]" , "What's significant about this paper: [paper]?", "What does [topic] mean?", etc. It should be a single question to collect useful information on. Do not propose multiple questions at the same time. Do not mention anything else. 
        4) The next question should not be too hard since the internet and I may not contain the full answer in a single article or have learned enough information to complete it yet. 
        5) The next question should be novel and interesting based on my current learning progress. I should look for rare and potentially useful pieces of information, upgrade my saved notes using better information, and discover new things. I should not be doing the same thing over and over again.
        6) I may sometimes need to repeat some questions or variations of the question if I need to collect more information to answer more difficult question. Only repeat questions if necessary. 
        7) I want to explore the world and discover new things. I don’t want to stay in my current state for too long. 
        8) Questions that require information beyond another person's ability to theoretically verify and reason if completed or correct should be avoided. For instance, "what else is there on the website?" and "what images and tables are on the website" are not ideal since they require visual confirmation from the screen. All the testing, coding, and asking other people questions should be avoided. Do not propose a question with these keywords. You should only respond in the format as described below:
        
        RESPONSE FORMAT: 
        Reasoning: Based on the information I listed above, do reasoning about what the next question should be. 
        Question: The next question. 
        
        Here’s an example response: 
        Reasoning: We know the we have a sword and we know there's fire, and fire lights things on fire. Therefore, we could try to make a firesword.
        Question: Could we make a firesword?
'''

        # TODO: This is optional, might be useful, but to focus on a system prompt of asking questions and answering questions.
        # System 2: this is a more scoped down version where we have the focus be on only answering questions -- reading and analyzing information & asking questions. No action items. 
        # The current above system 1 is better for self-driving labs type of work where there are going to be more tasks.

    def get_exploration_progress(self, completed_tasks, failed_tasks):
        # TODO: this should contain inventory of where we're at now and what files we have / memory stream
        return '''Completed tasks: None, Failed tasks: None'''

    def propose_next_question(self, skills, exploration_progress):
        '''
        This function decomposes a goal into tasks
        '''        
        user_prompt = ""
        observation = {
            "memory": f"Memory: {self.memory}\n\n",
            "root_folder_inventory": f"Root folder inventory: {self.files}\n\n",
            "saved_notes": f"Saved notes: {self.saved_notes}\n\n",
            "completed_tasks": f"Completed tasks so far: {self.completed_tasks}\n\n",
            "failed_tasks": f"Failed tasks that are too hard: {self.failed_tasks}\n\n",
        } # TODO: I don't think I'm even using saved notes
        user_prompt += "".join(observation.values())
        
        print("System prompt for generating curriculum: \n", self.system_prompt_automatic_curriculum, "\n User prompt: ", user_prompt)
        next_question_response = chat_openai(user_prompt, system_prompt=self.system_prompt_automatic_curriculum)[0]['content']
        print("Response: ", next_question_response)
        next_question = self.parse_message(next_question_response)["next_question"]
        return next_question

    def get_completed_tasks():
        pass

    def get_failed_tasks():
        pass

    def add_completed_task(self, task):
        self.completed_tasks.append(task)

    def add_failed_task(self, task):
        self.failed_tasks.append(task)

    def parse_message(self, message):
        question = ""
        for line in message.split("\n"):
            if line.startswith("Question:"):
                question = line[9:].strip()
        assert question, "Question not found in Curriculum Agent response"
        return {"next_question": question}

# hardcoded
files = '''train.csv - the training set
test.csv - the test set
data_description.txt - full description of each column, originally prepared by Dean De Cock but lightly edited to match the column names used here
sample_submission.csv - a benchmark submission from a linear regression on year and month of sale, lot square footage, and number of bedrooms
data_fields.txt - a brief version of what you'll find in the data description file.
'''
curriculum_agent = CurriculumAgent(goal=goal, files=files)

In [6]:
# print(curriculum_agent.completed_tasks)
# next_question = curriculum_agent.propose_next_question(skills=[], exploration_progress=[])
# print("next_question", next_question)

In [7]:
# next_question = "What is the structure and content of the dataset in the train.csv file?" # hardcoded for now

In [8]:
class SkillManager():
    # Additional skills to be added: await bot.findsProblem, await bot.findsInterestingInformation, await bot.findSolution or something like that

    def __init__(self):
        self.functions = [
            {
                "name": "read_file",
                "description": "Get the text from the file",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "file_name": {
                            "type": "string",
                            "description": "the complete file name and extension, e.g. abc.txt"
                        }
                    },
                    "required": ["file_name"],            
                }
            }, 
            {
                "name": "write_file",
                "description": "Write text to a file",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "file_name": {
                            "type": "string",
                            "description": "the complete file name and extension, e.g. abc.txt"
                        },
                        "text": {
                            "type": "string",
                            "description": "the text to write to the file"
                        }
                    },
                    "required": ["file_name", "text"],            
                }
            }, 
            {
                "name": "web_search",
                "description": "Given a prompt, I'll return back my web search information about it",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "Tell me what to search about"
                        }
                    },
                    "required": ["query"],            
                }
            },
            {
                "name": "think",
                "description": "Given a prompt, I'll return back my thoughts on it.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "Tell me what to think about"
                        }
                    },
                    "required": ["query"],            
                }
            },
            {
                "name": "num_children",
                "description": "This paper talks about how many girls and boys are in this family",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": [],
                },
            },
            
            # {
            #     "name": "content_in_data_txt",
            #     "description": "I know the information in data.txt",
            #     "parameters": {
            #         "type": "object",
            #         "properties": {},
            #         "required": [],
            #     },
            # }
            # {'name': 'columns_and_descriptions', 'description': "The question was what are the columns and their descriptions in the training set and the test set. The answer is that the columns and their descriptions can be found in the 'data_description.txt' file using the 'read_file' function. The columns and their descriptions in both the training set and the test set include information about the type of dwelling, zoning classification, lot size, road and alley access, property shape and flatness, utilities available, and lot configuration.", 'parameters': {'type': 'object', 'properties': {}, 'required': []}}
        ]

        self.available_functions = {
            "read_file": self.read_file,
            "write_file": self.write_file,
            "web_search": self.web_search,
            "think": self.think,
            "num_children": self.num_children,
            # "columns_and_descriptions": self.columns_and_descriptions
            # "content_in_data_txt": self.content_in_data_txt,
        }

        # Currently not much utility, but if you needed to trace back to a function or fact it got wrong, this can help.
        # TODO: honestly, not really sure what to do with this. I was thinking seeing the entire message history would be a good way to trace back, but we already have the original message prompt
        self.function_history = [
            "given",
            "given",
            "given",
            "given",
            "debug",
        ]

        self.files = [
            "train.csv - the training set",
            "test.csv - the test set",
            "data_description.txt - full description of each column, originally prepared by Dean De Cock but lightly edited to match the column names used here",
            "sample_submission.csv - a benchmark submission from a linear regression on year and month of sale, lot square footage, and number of bedrooms",
            "data_fields.txt - a brief version of what you'll find in the data description file.",
        ]

    def retrieve_skills(self, task, execution_feedback):
        # For sake of simplicity, use recency for now (later relevancy and importance can be added)
        func_name_description_list = [str((func["name"], func["description"])) + '\n' for func in self.functions]
        return func_name_description_list

    # Helper function: be able to create a function with a dynamic name and return value
    def create_skill_function(self, function_name, return_value):
        def dynamic_method(self):
            return return_value
        # Bind the function to the instance as a method
        bound_method = types.MethodType(dynamic_method, self)
        setattr(self, function_name, bound_method)
        # Add the method to available functions
        self.available_functions[function_name] = bound_method

    # Core function: adding a new skill requires an original task, a validated answer, and a message history
    def add_skill(self, task, validated_answer, methods_prompt):
        # TODO: wait until the action agent generates a function because maybe you only need to write a description of the input function instead of task and validated answer.
        # create_function_description_system_prompt = f'''You are a helpful assistant that writes a description of the given '''

        print("Adding skills! ", task, validated_answer, methods_prompt)

        create_function_system_prompt = f'''You are a helpful assistant. Note that a function is a concept here, which means that a function is called when a user wants more information about the description which should contain the both question and answer, and which will return the reasoning behind the answer if called. Your goal is to create a JSON description for a function where the name is a unique short function name and the description contains both the task and the answer. 
        
        You will receive this information:
        Original task or question: ...
        Answer: ...

        Do not use any of these function names: {self.available_functions.keys()}

        Your output should be in the following format if function requires arguments:
        ```json
        {{
            "name": "<function_name>",
            "description": "<insert task and answer>",
            "parameters": {{
                "type": "object",
                "properties": {{
                    "<insert arg 1 name>": {{
                        "type": "<insert arg 1 type>",
                        "description": "<insert arg 1 description>"
                    }},
                    ...
                }},
                "required": ["<insert arg name if needed>", ...]         
            }}
        }}
        ```

        Good example output:
        ```json
        {{
            "name": "num_dogs_in_bens_family",
            "description": "The question was how many dogs are in the family. Ben said that he has 2 dogs in his family.",
            "parameters": {{
                "type": "object",
                "properties": {{}},
                "required": []
            }}
        }}
        ```

        Ensure the response can be parsed by Python "json.loads", e.g.: no trailing commas, no single quotes, etc. This is important.
        '''

        create_function_description_prompt = f'''
        Original task or question: {task}
        Answer: {validated_answer}
        '''
        res, messages = chat_openai(prompt=create_function_description_prompt, system_prompt=create_function_system_prompt, verbose=True)
        res

        try:
            # Load the function description
            func_description = json.loads(res['content'])
            print("func_description: ", func_description)

            # Create the function as a method of skill_manager
            self.create_skill_function(func_description['name'], methods_prompt + "\n" + validated_answer)
            print(exec(f"self.{func_description['name']}"))

            # Add function to function description list
            self.functions.append(func_description)

            # Record the messages history
            # self.function_history.append(methods_prompt + "\n" + validated_answer)

            print("COMPLETE!")
            
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            return

        return func_description

    # Below are given functions or dummy functions
    def read_file(self, file_name):
        '''Get the text from the file'''
        file_path = f'{root_file_directory}/{file_name}'
        with open(file_path, 'r') as file:
            text = file.read()[:2000]
        return text
    
    def write_file(self, file_name, text):
        '''Write the given text to the file.'''
        file_path = f'{root_file_directory}/{file_name}'
        with open(file_path, 'w') as file:  # 'w' is the mode for writing to a file
            file.write(text)  # Write the entire text to the file
        return text

    def web_search(self, query):
        # Temporarily just return search info from the user
        return input(f"Search for {query} on the web: ")
        
    def think(self, query):
        return chat_openai(query + " Think step by step.", verbose=True)[0]['content']
    
    # Demo of how to add plain just fact / info skills that gets added as just natural language?
    def content_in_data_txt(self):
        return '''The data.txt file contains the following information: File descriptions
train.csv - the training set
test.csv - the test set
data_description.txt - full description of each column, originally prepared by Dean De Cock but lightly edited to match the column names used here
sample_submission.csv - a benchmark submission from a linear regression on year and month of sale, lot square footage, and number of bedrooms
data_fields.txt - a brief version of what you'll find in the data desc'ription file.'''

    def num_children(self):
        return '''There are 2 boys, and 8 girls'''
    
    # MANUALLY ADDED FUNCTIONS FOR TESTING
    def columns_and_descriptions():
        return '''The question was what are the columns and their descriptions in the training set and the test set. The answer is that the columns and their descriptions can be found in the 'data_description.txt' file using the 'read_file' function. The columns and their descriptions in both the training set and the test set include information about the type of dwelling, zoning classification, lot size, road and alley access, property shape and flatness, utilities available, and lot configuration.'''
    
skill_manager = SkillManager()
print("Available functions: \n", skill_manager.available_functions, "\n\nFunctions", skill_manager.functions)
# Unit test to ensure that the skill manager's basic skills work
skill_manager.write_file("hello_world.txt", "hello_world")
assert(os.path.isfile(f'{root_file_directory}/hello_world.txt'))
read_file_result = skill_manager.read_file("hello_world.txt")
assert(read_file_result == "hello_world")
web_search_result = skill_manager.web_search("Do penguins fly?")
assert(len(web_search_result) > 0 and type(web_search_result) == str)
# think_result = skill_manager.think("Do penguins fly?") # Commenting out just for now since it costs to test
# assert(len(think_result) > 0 and type(think_result) == str)

Available functions: 
 {'read_file': <bound method SkillManager.read_file of <__main__.SkillManager object at 0x000001B635AF7610>>, 'write_file': <bound method SkillManager.write_file of <__main__.SkillManager object at 0x000001B635AF7610>>, 'web_search': <bound method SkillManager.web_search of <__main__.SkillManager object at 0x000001B635AF7610>>, 'think': <bound method SkillManager.think of <__main__.SkillManager object at 0x000001B635AF7610>>, 'num_children': <bound method SkillManager.num_children of <__main__.SkillManager object at 0x000001B635AF7610>>} 

Functions [{'name': 'read_file', 'description': 'Get the text from the file', 'parameters': {'type': 'object', 'properties': {'file_name': {'type': 'string', 'description': 'the complete file name and extension, e.g. abc.txt'}}, 'required': ['file_name']}}, {'name': 'write_file', 'description': 'Write text to a file', 'parameters': {'type': 'object', 'properties': {'file_name': {'type': 'string', 'description': 'the complete fil

In [9]:
# skills = skill_manager.retrieve_skills(next_question, execution_feedback=None)
# print(skills)

In [10]:
# # Start of demo of adding a skill
# # Adding a skill, given a task and validated answer
# task = "Who let the dogs out?"
# content = 'It was Joe. he peed and then joe had to go grab him. olivia said no, but joe ultimately did it.'

In [11]:
# skill_manager.whoLetTheDogsOut()

In [12]:
# skill_manager.add_skill(task, content, "blah blah blah")
# # End of demo of adding a skill

In [13]:
# # Using chat completions to use the skills
# res, messages = chat_openai(prompt="Write a file named kevin.txt and write the name Kevin Huang", functions=skill_manager.functions, available_functions=skill_manager.available_functions, verbose=True)
# res

In [14]:
class ActionAgent():
    # TODO: does there need to be an action agent generating the steps? Or does there need to be a separate execution agent running the prompt? Or can the action agent be the execution agent?
    # TODO: can a critic agent really check if the output is correct? Or can they only check if that aligns with expectation? Otherwise, the critic will have to check the line of content values to make sure the reasoning is sound, which is still doable, but the extent that another critic can check is limited. I guess it's just to make sure the reasoning is sound.

    def __init__(self):
        pass

    def generate_function_callable_prompt(self, task,
                methods_prompt,
                execution_feedback,
                execution_errors,
                critique,
                skills,
                skill_manager):
        generate_function_callable_system_prompt = f'''You are a helpful assistant and a first-rate problem solver. Given a task or question, your goal is to list out the steps to solve that task or question given your skills and reasoning. Ultimately, your output should be able to be followed by a human limited by the skills and knowledge given, and another human should be able to check that human's output to see if it's correct and reasonable. Note that the functions asked for may sometimes already be called and the information from the function that you need is already in the prompt, so read carefully. Note that you DO NOT have access to run any code, you can only read, think, and write about the existing skills and knowledge.

        You will be given this information:
        Task or question: ...
        Skills & knowledge: You only have these skills and pieces of knowledge in the format of (function name which represents a concept / ability, a description of that concept and ability that you can invoke) to help you write the steps to achieve this task or answer this question. You can call a function (sometimes with or without arguments) to just get more information too.
        Files: ... 
        Current state solution: ...
        Execution feedback: ...
        Execution errors: ...
        Critique: ...'''

        user_prompt = f'''Task or question: {task}
        Skills & knowledge: {skills}
        Files: {skill_manager.files}
        Current state solution: {methods_prompt}
        Execution feedback: {execution_feedback}
        Execution errors: {execution_errors}
        Critique: {critique}'''

        agent_methods_feedback, agent_methods_errors = self.function_call(generate_function_callable_system_prompt, user_prompt, skill_manager)

        # TODO: Add error handling for if the agent_methods_feedback is empty and agent_methods_errors exists
        print("agent_methods_feedback", agent_methods_feedback, "agent_methods_errors", agent_methods_errors)

        return f'Task or question: {task} \nInstructions: ' + agent_methods_feedback['content']
    
    def function_call(self, system_prompt, methods_prompt, skill_manager):
        print()
        try:
            response_message, messages = chat_openai(system_prompt=system_prompt, prompt=methods_prompt, functions=skill_manager.functions, available_functions=skill_manager.available_functions,verbose=True)
        except Exception as e:
            return "", e
        return response_message, None
    
action_agent = ActionAgent()
methods_prompt = None
execution_feedback = None
execution_errors = None
critique = None
success = False

In [15]:
# next_question = "How many children are there?" # hardcoded for now
# skills = skill_manager.retrieve_skills(next_question, execution_feedback=None)
# print(skills)

In [16]:
# methods_prompt = action_agent.generate_function_callable_prompt(next_question, methods_prompt, execution_feedback, execution_errors, critique, skills, skill_manager)
# print("Methods prompt:\n", methods_prompt)

In [17]:
# print(methods_prompt)

In [18]:
# methods_prompt_unreasonable = '''Task or question: How many parents are there? 
# Steps to find out how many parents there are:

# 1. Use the function "num_children" to determine the number of boys and girls in the family.
# 2. The function "num_children" will return the count of boys and girls separately.
# 3. Add the count of boys and girls together to get the total number of children.

# Based on the provided skills and knowledge, the function "num_children" appears to be the most relevant for determining the number of parents.'''

In [19]:
class ExecutionAgent():
    # Ultimately the answer will go into Description or be "returned" with the description being condensed into a tldr and the methods_prompt will be added to history

    def function_call(self, methods_prompt, skill_manager):
        system_prompt = f'''You are a helpful assistant. Your goal is to execute the given instructions and output the complete answer to the question. If the instructions don't seem reasonable or you cannot get to the complete answer, then you should give feedback on why you couldn't do it and what you tried. 

        You will be given this information:
        Instructions: ...
'''
        
        try:
            response_message, messages = chat_openai(system_prompt=system_prompt, prompt=methods_prompt, functions=skill_manager.functions, available_functions=skill_manager.available_functions,verbose=True)
            conclusion = response_message['content'] 

            # # Assuming response_message['content'] is a string that contains 'Conclusion:'
            # match = re.search(r'Conclusion:(.*)', response_message['content'], re.DOTALL)
            # if match:
            #     conclusion = match.group(1).strip()
            # else:
            #     conclusion = response_message['content']  # or some default value, in case "Conclusion:" isn't found
        except Exception as e:
            return "", e
        return conclusion, None
execution_agent = ExecutionAgent()

In [20]:
# execution_feedback, execution_errors = execution_agent.function_call(methods_prompt, skill_manager)
# print("Execution feedback: ", execution_feedback, "\nExecution errors: ", execution_errors)

In [21]:
# execution_feedback

In [22]:
class CriticAgent():
    # TODO: Perhaps include the skills to the critic so the critic knows the facts to check if this makes sense or not

    def check_task_success(self, task, methods_prompt, execution_feedback, skills, skill_manager):
        system_prompt = '''You are an assistant that assesses my progress of research and provides useful guidance. 
        
        You are required to evaluate if I have provided a complete answer to the question. Providing more information and exceeding the task requirements is also considered a success while failing to meet them or not completely answering the question requires you to provide critique to help me improve.

        I will give you the following information:
        Skills & knowledge: My skills and pieces of information and knowledge I have. 
        Files: My files.
        Task or question: The question I need to answer.
        Answer: My current answer.
        Approach: My reasoning of how I got to my answer.

        You should only respond in JSON format as described below:
        {
            "reasoning": "reasoning",
            "success": boolean,
            "critique": "critique",
        }
        Ensure the response can be parsed by Python "json.loads", e.g.: no trailing commas, no single quotes, etc.

        Here are some examples:
        INPUT:
        Task or question: What is the distribution of the sale prices in the dataset?
        Answer: To determine the distribution of the sale prices in the dataset, we can follow these steps:\n\n1. Read the dataset file "train.csv" using the `read_file` function.\n2. Extract the column containing the sale prices from the dataset.\n3. Calculate the frequency of each unique sale price in the dataset.\n4. Sort the unique sale prices in ascending order.\n5. Create a histogram or bar chart to visualize the distribution of the sale prices.\n6. Optionally, you can also calculate summary statistics such as mean, median, and standard deviation of the sale prices.\n\nLet\'s start by reading the dataset file "train.csv".
        Approach: Task or question: What is the distribution of the sale prices in the dataset? \nInstructions: To determine the distribution of the sale prices in the dataset, you can follow these steps:\n\n1. Read the dataset file "train.csv" using the `read_file` function.\n2. Extract the column containing the sale prices from the dataset.\n3. Calculate the frequency of each unique sale price in the dataset.\n4. Sort the unique sale prices in ascending order.\n5. Create a histogram or bar chart to visualize the distribution of the sale prices.\n6. Optionally, you can also calculate summary statistics such as mean, median, and standard deviation of the sale prices.\n\nPlease note that the specific implementation details may vary depending on the programming language and libraries you are using.

        RESPONSE:
        {
            "reasoning": The reasoning to get to the answer makes sense, but I don't see an answer for what the actual distirbution of the sale price is.,
            "success": False,
            "critique": The answer only tells us how to get the distribution is, but does not tell us what the actual distribution.
        }
        '''
        # Tactic: add few shot examples
        # Here are some examples:
        # INPUT:
        # Task or question: How 
        # Answer: 
        # Approach: 

        # RESPONSE:
        # {
        #     "reasoning": "reasoning",
        #     "success": boolean,
        #     "critique": "critique",
        # }

        user_prompt = f'''Skills & knowledge: {skills}
        Files: {skill_manager.files}
        Task or question: {task}
        Answer: {execution_feedback}
        Approach: {methods_prompt}'''

        response_message, messages = chat_openai(system_prompt=system_prompt, prompt=user_prompt, verbose=True)

        response_json = json.loads(response_message['content'])
        success = response_json['success']
        reasoning = response_json['reasoning']
        critique = response_json['critique']

        # lines = response_message['content'].split("\n")
        # for line in lines:
        #     # Strip whitespace for accurate matching
        #     line = line.strip()
            
        #     # Check if the line starts with the known titles and parse accordingly
        #     if line.startswith('Reasoning:'):
        #         critique = line[len('Reasoning:'):].strip()
        #     elif line.startswith('Task is reasonably answered:'):
        #         success = line[len('Task is reasonably answered:'):].strip().lower()

        print("System prompt: ", system_prompt, "\nUser prompt: ", user_prompt, "\nResponse: ", response_message['content'])
        return success, critique
critic_agent = CriticAgent()

In [23]:
# success, critique = critic_agent.check_task_success(
#                 next_question, methods_prompt,  execution_feedback
#             )
# print("Success: ", success, "\nCritique: ", critique)

In [24]:
# Instantiate only once! 
skill_manager = SkillManager()

In [25]:
# START HERE FOR NEXT ITERATIONS: Save the current iteration's state after this cycle in case we need to revert
skill_manager_copy = copy.deepcopy(skill_manager)
curriculum_agent_copy = copy.deepcopy(curriculum_agent)

In [26]:
# REVERT BY STARTING HERE, uncomment the below and run
# skill_manager = copy.deepcopy(skill_manager_copy)
# curriculum_agent = copy.deepcopy(curriculum_agent_copy)

In [27]:
print("curriculum_agent.completed_tasks", curriculum_agent.completed_tasks, "curriculum_agent.failed_tasks", curriculum_agent.failed_tasks)

curriculum_agent.completed_tasks [] curriculum_agent.failed_tasks []


In [28]:
print("curriculum_agent.completed_tasks", curriculum_agent.completed_tasks)
next_question = curriculum_agent.propose_next_question(skills=skill_manager.retrieve_skills(task="", execution_feedback=""), exploration_progress=[])
print("next_question", next_question)

curriculum_agent.completed_tasks []
System prompt for generating curriculum: 
 You are a helpful assistant that asks questions to help me decide the next immediate question to answer on the computer. My ultimate goal is to discover as many useful pieces of information as possible, answer as many questions as possible, and become the best researcher in the world.

        Goal: 
Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

Goal
It is your job to predict the sales price for each house. For each Id in the test set, you must predict the value of the SalePrice v

Response:  Reasoning: Since we have the training set and the test set, it would be helpful to understand the structure and content of these datasets before we start analyzing them. 

Question: What are the columns and their descriptions in the training set and the test set?
next_question What are the columns and their descriptions in the training set and the test set?


In [29]:
print("\nCurrent skills\n", skill_manager.retrieve_skills(task="", execution_feedback=None))


Current skills
 ["('read_file', 'Get the text from the file')\n", "('write_file', 'Write text to a file')\n", '(\'web_search\', "Given a prompt, I\'ll return back my web search information about it")\n', '(\'think\', "Given a prompt, I\'ll return back my thoughts on it.")\n', "('num_children', 'This paper talks about how many girls and boys are in this family')\n"]


In [30]:
print(skill_manager.functions)
print(skill_manager.available_functions)

[{'name': 'read_file', 'description': 'Get the text from the file', 'parameters': {'type': 'object', 'properties': {'file_name': {'type': 'string', 'description': 'the complete file name and extension, e.g. abc.txt'}}, 'required': ['file_name']}}, {'name': 'write_file', 'description': 'Write text to a file', 'parameters': {'type': 'object', 'properties': {'file_name': {'type': 'string', 'description': 'the complete file name and extension, e.g. abc.txt'}, 'text': {'type': 'string', 'description': 'the text to write to the file'}}, 'required': ['file_name', 'text']}}, {'name': 'web_search', 'description': "Given a prompt, I'll return back my web search information about it", 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Tell me what to search about'}}, 'required': ['query']}}, {'name': 'think', 'description': "Given a prompt, I'll return back my thoughts on it.", 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'de

In [31]:
# Attempting to build 1 cycle with all agents
# next_question = "Search up how many penguins there are in Kevin Huang's house. We don't need an exact or true number."
# next_question = "What is the total number of penguins and children that we know about from our functions?" # hardcoded for now
num_rounds = 2
num_tasks = 1
print("next_question", next_question)

next_question What are the columns and their descriptions in the training set and the test set?


In [32]:
for task_idx in range(num_tasks):
    methods_prompt = None
    execution_feedback = None
    execution_errors = None
    critique = None
    success = False
    for i in range (num_rounds):
        print("\nTask: \n", next_question)
        
        skills = skill_manager.retrieve_skills(next_question, execution_feedback=None)
        print("\nSkill manager output:\n", skills)

        print("\nStarting action agent")
        action_agent = ActionAgent()
        methods_prompt = action_agent.generate_function_callable_prompt(next_question, methods_prompt, execution_feedback, execution_errors, critique, skills, skill_manager)
        print("\nAction agent output:\n", methods_prompt)

        print("\nStarting execution agent")
        execution_agent = ExecutionAgent()
        execution_feedback, execution_errors = execution_agent.function_call(methods_prompt, skill_manager)
        print("\nExecution agent output: ", execution_feedback, "\nExecution errors: ", execution_errors)

        if execution_errors:
            continue

        print("\nStarting critic agent")
        critic_agent = CriticAgent()
        success, critique = critic_agent.check_task_success(next_question, methods_prompt, execution_feedback, skills, skill_manager)
        print("Critic agent output", "\nSuccess: ", success, "\nCritique: ", critique)

        if success:
            break
    if success:
        print("\nBefore adding skill\n", skill_manager.retrieve_skills("", execution_feedback=None))
        skill_manager.add_skill(next_question, execution_feedback, methods_prompt)
        print("\nAfter adding skill\n", skill_manager.retrieve_skills("", execution_feedback=None))
        curriculum_agent.add_completed_task(next_question)
    else:
        curriculum_agent.add_failed_task(next_question)


Task: 
 What are the columns and their descriptions in the training set and the test set?

Skill manager output:
 ["('read_file', 'Get the text from the file')\n", "('write_file', 'Write text to a file')\n", '(\'web_search\', "Given a prompt, I\'ll return back my web search information about it")\n', '(\'think\', "Given a prompt, I\'ll return back my thoughts on it.")\n', "('num_children', 'This paper talks about how many girls and boys are in this family')\n"]

Starting action agent

Prompt messages:  [{'role': 'system', 'content': "You are a helpful assistant and a first-rate problem solver. Given a task or question, your goal is to list out the steps to solve that task or question given your skills and reasoning. Ultimately, your output should be able to be followed by a human limited by the skills and knowledge given, and another human should be able to check that human's output to see if it's correct and reasonable. Note that the functions asked for may sometimes already be calle

Completion info:  {
  "id": "chatcmpl-8HjDTKWoqMvZS7CPZY6XO6RoXH1q6",
  "object": "chat.completion",
  "created": 1699236079,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "read_file",
          "arguments": "{\n  \"file_name\": \"data_description.txt\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 703,
    "completion_tokens": 18,
    "total_tokens": 721
  }
}
Entering function call:  {
  "name": "read_file",
  "arguments": "{\n  \"file_name\": \"data_description.txt\"\n}"
}
Function call output:  MSSubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STOR

In [33]:
# def generate_how(goal, next_task, task_history = []):
#     '''
#     This function creates the function for the task and "how" for the task
#     '''
#     gen_how_prompt = f'''You are a first-rate problem solver. 
    
#     Your task is to operationalize the following action item as a code function: {next_task}
#     Only use action functions that you already have and other normal code. You must solve the problem using code, action functions that you create and can build upon, and words.

#     Write a function for how to achieve this task using the action functions that you have. If you cannot achieve this task with the action functions that you have, then respond with "I don't know because <your reasoning>". If you can achieve this task, then please first reason on how to do this, and then conclude with the task function in this format: "Task function: <insert task function name>(<insert args>): <insert action functions to use>".

#     Rules:
#     - In the concluding list of steps, pretend that the reader can only see the list of steps and not your reasoning so make sure the list of steps is clear.
#     - Make sure that the next best action requires is a small step and reasonable enough for a high schooler to figure out how to do it.

#     Action functions that you have:
#     1) think(prompt) -- thinking out loud given the prompt
#     2) web_search(query) -- web searching for query and returning some information
#     3) read(file) -- reading file, but the first 4000 chars
#     4) write(file) -- writing to file

#     Action functions that you are constrained to currently not have:
#     1) code() -- writing code
#     2) see() -- look at visualizations

#     Current inventory:
#     File descriptions
    # train.csv - the training set
    # test.csv - the test set
    # data_description.txt - full description of each column, originally prepared by Dean De Cock but lightly edited to match the column names used here
    # sample_submission.csv - a benchmark submission from a linear regression on year and month of sale, lot square footage, and number of bedrooms
    # data_fields.txt - a brief version of what you'll find in the data description file.
#     '''
#     print("Prompt for generating how to execute: \n", gen_how_prompt)
#     next_step = chat_openai(gen_how_prompt)[0]
#     return next_step

# gen_function = generate_how(goal, next_task, [])
# print(gen_function)